In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!mkdir ~/.kaggle

!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:

!kaggle competitions download -c isic-2024-challenge


100% 2.00G/2.00G [00:16<00:00, 92.9MB/s]
100% 2.00G/2.00G [00:16<00:00, 128MB/s] 


In [3]:
!pip install --upgrade timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.9 MB/s eta 0:00:00


In [4]:
!pip install pydicom wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.2 MB/s eta 0:00:00


In [5]:
!pip install -U libauc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 7.1 MB/s eta 0:00:00


In [6]:
!unzip -q /content/isic-2024-challenge.zip

In [7]:
# !kaggle datasets download -d fabiangrger/isic-2024-challenge-selfclean-scores


In [8]:
# !unzip -q /content/isic-2024-challenge-selfclean-scores.zip

# Train an ImageNet to classify the ISIC 2024 Data / Generate OOF Preds
* This is a (detuned) public version of my notebook intended to demonstrate how to generate OOF preds for use by LGBM
* Idea is to show how this works (it will probably not produce a rocking LB score)
* This notebook will predicts on test using an average of all folds
* It will also output the models / oof_predictions.csv for easy integration with LGBM / similar...

* For an example of how to use see https://www.kaggle.com/code/richolson/isic-2024-borrowed-179lb-tabular-oof-imagenet/notebook

* Note: There is a bunch of redundant code here (intent is to make sharing with other notebooks easier)

In [1]:
import os
import random
import time
from datetime import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
import polars as pl
from libauc.sampler import DualSampler
from libauc.losses import pAUCLoss
from libauc.optimizers import SOPAs, SOPA
import cv2
from PIL import Image
import io
import h5py
import math

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import timm

from sklearn.metrics import hamming_loss, f1_score, roc_curve, auc, classification_report
from sklearn.preprocessing import binarize
from sklearn.model_selection import GroupShuffleSplit

import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import GroupKFold, StratifiedGroupKFold

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.16 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


# Misc. Setup

In [2]:
# Set up device and random seed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")

random_seed = 42
random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

#number of epochs to train for
num_epochs = 50

#train entire model vs. just the classifier
freeze_base_model = False  #didn't get good results

# if this is set to true - full model is only generated as part of scoring (quick_train_record_count used)
# this saves GPU quota - but saved model won't reflect what was scored...
full_train_only_when_scoring = False  #must be False to save full model!
quick_train_record_count = 50000   #need to get at least some positive cases even for test run

Using device: cuda
GPU: Tesla T4
Number of GPUs: 1


# Load meta - and split folds
* Maintain consistency with tabular data in other notebooks

In [3]:
bad_ids = ['ISIC_0091661', 'ISIC_0157300', 'ISIC_0164004', 'ISIC_0225902',
       'ISIC_0247991', 'ISIC_0253221', 'ISIC_0275821', 'ISIC_0276162',
       'ISIC_0321282', 'ISIC_0512487', 'ISIC_0573025', 'ISIC_0749379',
       'ISIC_0887823', 'ISIC_1142893', 'ISIC_1180656', 'ISIC_1194950',
       'ISIC_1280179', 'ISIC_1334224', 'ISIC_1338006', 'ISIC_1443812',
       'ISIC_1488609', 'ISIC_1618438', 'ISIC_1716141', 'ISIC_1755348',
       'ISIC_1882290', 'ISIC_2066646', 'ISIC_2082383', 'ISIC_2153489',
       'ISIC_2185868', 'ISIC_2301755', 'ISIC_2325643', 'ISIC_2326801',
       'ISIC_2501464', 'ISIC_2563846', 'ISIC_2592061', 'ISIC_2649560',
       'ISIC_2842612', 'ISIC_3019770', 'ISIC_3245832', 'ISIC_3355799',
       'ISIC_3606755', 'ISIC_3673016', 'ISIC_3856578', 'ISIC_3902330',
       'ISIC_3905526', 'ISIC_3915836', 'ISIC_3970343', 'ISIC_4029206',
       'ISIC_4172573', 'ISIC_4244859', 'ISIC_4435163', 'ISIC_4590578',
       'ISIC_4628194', 'ISIC_4723477', 'ISIC_4794318', 'ISIC_4837618',
       'ISIC_4859161', 'ISIC_4884516', 'ISIC_4992507', 'ISIC_5129222',
       'ISIC_5374420', 'ISIC_5446672', 'ISIC_5644802', 'ISIC_5678455',
       'ISIC_5687461', 'ISIC_5873888', 'ISIC_5874842', 'ISIC_5938732',
       'ISIC_5990814', 'ISIC_6021059', 'ISIC_6145237', 'ISIC_6233830',
       'ISIC_6265900', 'ISIC_6290217', 'ISIC_6303557', 'ISIC_6347423',
       'ISIC_6415548', 'ISIC_6443962', 'ISIC_6505439', 'ISIC_6731439',
       'ISIC_6757661', 'ISIC_6794549', 'ISIC_6796625', 'ISIC_6838918',
       'ISIC_6931102', 'ISIC_7028157', 'ISIC_7348810', 'ISIC_7358578',
       'ISIC_7386083', 'ISIC_7445245', 'ISIC_7478620', 'ISIC_7546705',
       'ISIC_7805616', 'ISIC_8091604', 'ISIC_8182531', 'ISIC_8278020',
       'ISIC_8379868', 'ISIC_8537711', 'ISIC_8644028', 'ISIC_8653720',
       'ISIC_8755758', 'ISIC_9118564', 'ISIC_9156598', 'ISIC_9342841',
       'ISIC_9458222', 'ISIC_9476302', 'ISIC_9546926', 'ISIC_9611761',
       'ISIC_9645582', 'ISIC_9675639', 'ISIC_9713969', 'ISIC_9758190',
        'ISIC_0235999', 'ISIC_0633292', 'ISIC_1137305', 'ISIC_1203239',
       'ISIC_1243149', 'ISIC_1449578', 'ISIC_1642217', 'ISIC_1939971',
       'ISIC_1992066', 'ISIC_1997122', 'ISIC_3113900', 'ISIC_3809050',
       'ISIC_3939743', 'ISIC_4368177', 'ISIC_4704252', 'ISIC_4711591',
       'ISIC_5063101', 'ISIC_5105473', 'ISIC_5290013', 'ISIC_5859200',
       'ISIC_6050068', 'ISIC_7083114', 'ISIC_7134832', 'ISIC_7210490',
       'ISIC_8397035', 'ISIC_8644948', 'ISIC_8829281', 'ISIC_8897434',
       'ISIC_9183347', 'ISIC_9200535', 'ISIC_9441116', 'ISIC_9500444']

In [4]:
num_cols = [
    'age_approx',                        # Approximate age of patient at time of imaging.
    'clin_size_long_diam_mm',            # Maximum diameter of the lesion (mm).+
    'tbp_lv_A',                          # A inside  lesion.+
    'tbp_lv_Aext',                       # A outside lesion.+
    'tbp_lv_B',                          # B inside  lesion.+
    'tbp_lv_Bext',                       # B outside lesion.+
    'tbp_lv_C',                          # Chroma inside  lesion.+
    'tbp_lv_Cext',                       # Chroma outside lesion.+
    'tbp_lv_H',                          # Hue inside the lesion; calculated as the angle of A* and B* in LAB* color space. Typical values range from 25 (red) to 75 (brown).+
    'tbp_lv_Hext',                       # Hue outside lesion.+
    'tbp_lv_L',                          # L inside lesion.+
    'tbp_lv_Lext',                       # L outside lesion.+
    'tbp_lv_areaMM2',                    # Area of lesion (mm^2).+
    'tbp_lv_area_perim_ratio',           # Border jaggedness, the ratio between lesions perimeter and area. Circular lesions will have low values; irregular shaped lesions will have higher values. Values range 0-10.+
    'tbp_lv_color_std_mean',             # Color irregularity, calculated as the variance of colors within the lesion's boundary.
    'tbp_lv_deltaA',                     # Average A contrast (inside vs. outside lesion).+
    'tbp_lv_deltaB',                     # Average B contrast (inside vs. outside lesion).+
    'tbp_lv_deltaL',                     # Average L contrast (inside vs. outside lesion).+
    'tbp_lv_deltaLB',                    #
    'tbp_lv_deltaLBnorm',                # Contrast between the lesion and its immediate surrounding skin. Low contrast lesions tend to be faintly visible such as freckles; high contrast lesions tend to be those with darker pigment. Calculated as the average delta LB of the lesion relative to its immediate background in LAB* color space. Typical values range from 5.5 to 25.+
    'tbp_lv_eccentricity',               # Eccentricity.+
    'tbp_lv_minorAxisMM',                # Smallest lesion diameter (mm).+
    'tbp_lv_nevi_confidence',            # Nevus confidence score (0-100 scale) is a convolutional neural network classifier estimated probability that the lesion is a nevus. The neural network was trained on approximately 57,000 lesions that were classified and labeled by a dermatologist.+,++
    'tbp_lv_norm_border',                # Border irregularity (0-10 scale); the normalized average of border jaggedness and asymmetry.+
    'tbp_lv_norm_color',                 # Color variation (0-10 scale); the normalized average of color asymmetry and color irregularity.+
    'tbp_lv_perimeterMM',                # Perimeter of lesion (mm).+
    'tbp_lv_radial_color_std_max',       # Color asymmetry, a measure of asymmetry of the spatial distribution of color within the lesion. This score is calculated by looking at the average standard deviation in LAB* color space within concentric rings originating from the lesion center. Values range 0-10.+
    'tbp_lv_stdL',                       # Standard deviation of L inside  lesion.+
    'tbp_lv_stdLExt',                    # Standard deviation of L outside lesion.+
    'tbp_lv_symm_2axis',                 # Border asymmetry; a measure of asymmetry of the lesion's contour about an axis perpendicular to the lesion's most symmetric axis. Lesions with two axes of symmetry will therefore have low scores (more symmetric), while lesions with only one or zero axes of symmetry will have higher scores (less symmetric). This score is calculated by comparing opposite halves of the lesion contour over many degrees of rotation. The angle where the halves are most similar identifies the principal axis of symmetry, while the second axis of symmetry is perpendicular to the principal axis. Border asymmetry is reported as the asymmetry value about this second axis. Values range 0-10.+
    'tbp_lv_symm_2axis_angle',           # Lesion border asymmetry angle.+
    'tbp_lv_x',                          # X-coordinate of the lesion on 3D TBP.+
    'tbp_lv_y',                          # Y-coordinate of the lesion on 3D TBP.+
    'tbp_lv_z',                          # Z-coordinate of the lesion on 3D TBP.+
]

new_num_cols = [
    'lesion_size_ratio',                 # tbp_lv_minorAxisMM      / clin_size_long_diam_mm
    'lesion_shape_index',                # tbp_lv_areaMM2          / tbp_lv_perimeterMM **2
    'hue_contrast',                      # tbp_lv_H                - tbp_lv_Hext              abs
    'luminance_contrast',                # tbp_lv_L                - tbp_lv_Lext              abs
    'lesion_color_difference',           # tbp_lv_deltaA **2       + tbp_lv_deltaB **2 + tbp_lv_deltaL **2  sqrt
    'border_complexity',                 # tbp_lv_norm_border      + tbp_lv_symm_2axis
    'color_uniformity',                  # tbp_lv_color_std_mean   / tbp_lv_radial_color_std_max

    'position_distance_3d',              # tbp_lv_x **2 + tbp_lv_y **2 + tbp_lv_z **2  sqrt
    'perimeter_to_area_ratio',           # tbp_lv_perimeterMM      / tbp_lv_areaMM2
    'area_to_perimeter_ratio',           # tbp_lv_areaMM2          / tbp_lv_perimeterMM
    'lesion_visibility_score',           # tbp_lv_deltaLBnorm      + tbp_lv_norm_color
    'symmetry_border_consistency',       # tbp_lv_symm_2axis       * tbp_lv_norm_border
    'consistency_symmetry_border',       # tbp_lv_symm_2axis       * tbp_lv_norm_border / (tbp_lv_symm_2axis + tbp_lv_norm_border)

    'color_consistency',                 # tbp_lv_stdL             / tbp_lv_Lext
    'consistency_color',                 # tbp_lv_stdL*tbp_lv_Lext / tbp_lv_stdL + tbp_lv_Lext
    'size_age_interaction',              # clin_size_long_diam_mm  * age_approx
    'hue_color_std_interaction',         # tbp_lv_H                * tbp_lv_color_std_mean
    'lesion_severity_index',             # tbp_lv_norm_border      + tbp_lv_norm_color + tbp_lv_eccentricity / 3
    'shape_complexity_index',            # border_complexity       + lesion_shape_index
    'color_contrast_index',              # tbp_lv_deltaA + tbp_lv_deltaB + tbp_lv_deltaL + tbp_lv_deltaLBnorm

    'log_lesion_area',                   # tbp_lv_areaMM2          + 1  np.log
    'normalized_lesion_size',            # clin_size_long_diam_mm  / age_approx
    'mean_hue_difference',               # tbp_lv_H                + tbp_lv_Hext    / 2
    'std_dev_contrast',                  # tbp_lv_deltaA **2 + tbp_lv_deltaB **2 + tbp_lv_deltaL **2   / 3  np.sqrt
    'color_shape_composite_index',       # tbp_lv_color_std_mean   + bp_lv_area_perim_ratio + tbp_lv_symm_2axis   / 3
    'lesion_orientation_3d',             # tbp_lv_y                , tbp_lv_x  np.arctan2
    'overall_color_difference',          # tbp_lv_deltaA           + tbp_lv_deltaB + tbp_lv_deltaL   / 3

    'symmetry_perimeter_interaction',    # tbp_lv_symm_2axis       * tbp_lv_perimeterMM
    'comprehensive_lesion_index',        # tbp_lv_area_perim_ratio + tbp_lv_eccentricity + bp_lv_norm_color + tbp_lv_symm_2axis   / 4
    'color_variance_ratio',              # tbp_lv_color_std_mean   / tbp_lv_stdLExt
    'border_color_interaction',          # tbp_lv_norm_border      * tbp_lv_norm_color
    'border_color_interaction_2',
    'size_color_contrast_ratio',         # clin_size_long_diam_mm  / tbp_lv_deltaLBnorm
    'age_normalized_nevi_confidence',    # tbp_lv_nevi_confidence  / age_approx
    'age_normalized_nevi_confidence_2',
    'color_asymmetry_index',             # tbp_lv_symm_2axis       * tbp_lv_radial_color_std_max

    'volume_approximation_3d',           # tbp_lv_areaMM2          * sqrt(tbp_lv_x**2 + tbp_lv_y**2 + tbp_lv_z**2)
    'color_range',                       # abs(tbp_lv_L - tbp_lv_Lext) + abs(tbp_lv_A - tbp_lv_Aext) + abs(tbp_lv_B - tbp_lv_Bext)
    'shape_color_consistency',           # tbp_lv_eccentricity     * tbp_lv_color_std_mean
    'border_length_ratio',               # tbp_lv_perimeterMM      / pi * sqrt(tbp_lv_areaMM2 / pi)
    'age_size_symmetry_index',           # age_approx              * clin_size_long_diam_mm * tbp_lv_symm_2axis
    'index_age_size_symmetry',           # age_approx              * tbp_lv_areaMM2 * tbp_lv_symm_2axis
]

cat_cols = ['sex', 'anatom_site_general', 'tbp_tile_type', 'tbp_lv_location', 'tbp_lv_location_simple', 'attribution']
norm_cols = [f'{col}_patient_norm' for col in num_cols + new_num_cols]
special_cols = ['count_per_patient']

#norm_cols += image_cols
feature_cols = num_cols + cat_cols + special_cols + new_num_cols # norm_cols

print(f"...{len(feature_cols)} FEATURES IN TOTAL...")
print(f"\t– num_cols: {len(num_cols)}");
print(f"\t– new_num_cols: {len(new_num_cols)}");
print(f"\t– cat_cols: {len(cat_cols)}");
print(f"\t– norm_cols: {len(norm_cols)}");
print(f"\t– special_cols: {len(special_cols)}");

...83 FEATURES IN TOTAL...
	– num_cols: 34
	– new_num_cols: 42
	– cat_cols: 6
	– norm_cols: 76
	– special_cols: 1


In [5]:

def read_data(path):
    return (
        pl.read_csv(path)
        .with_columns(
            pl.col('age_approx').cast(pl.Utf8).replace('NA', np.nan).cast(pl.Float64),
        )
        .with_columns(
            pl.col(pl.Float64).fill_nan(pl.col(pl.Float64).median()), # You may want to impute test data with train
        )
        .with_columns(
            lesion_size_ratio              = pl.col('tbp_lv_minorAxisMM') / pl.col('clin_size_long_diam_mm'),
            lesion_shape_index             = pl.col('tbp_lv_areaMM2') / (pl.col('tbp_lv_perimeterMM') ** 2),
            hue_contrast                   = (pl.col('tbp_lv_H') - pl.col('tbp_lv_Hext')).abs(),
            luminance_contrast             = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs(),
            lesion_color_difference        = (pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2).sqrt(),
            border_complexity              = pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_symm_2axis'),
            color_uniformity               = pl.col('tbp_lv_color_std_mean') / (pl.col('tbp_lv_radial_color_std_max') + err),
        )
        .with_columns(
            position_distance_3d           = (pl.col('tbp_lv_x') ** 2 + pl.col('tbp_lv_y') ** 2 + pl.col('tbp_lv_z') ** 2).sqrt(),
            perimeter_to_area_ratio        = pl.col('tbp_lv_perimeterMM') / pl.col('tbp_lv_areaMM2'),
            area_to_perimeter_ratio        = pl.col('tbp_lv_areaMM2') / pl.col('tbp_lv_perimeterMM'),
            lesion_visibility_score        = pl.col('tbp_lv_deltaLBnorm') + pl.col('tbp_lv_norm_color'),
            combined_anatomical_site       = pl.col('anatom_site_general') + '_' + pl.col('tbp_lv_location'),
            symmetry_border_consistency    = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border'),
            consistency_symmetry_border    = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border') / (pl.col('tbp_lv_symm_2axis') + pl.col('tbp_lv_norm_border')),
        )
        .with_columns(
            color_consistency              = pl.col('tbp_lv_stdL') / pl.col('tbp_lv_Lext'),
            consistency_color              = pl.col('tbp_lv_stdL') * pl.col('tbp_lv_Lext') / (pl.col('tbp_lv_stdL') + pl.col('tbp_lv_Lext')),
            size_age_interaction           = pl.col('clin_size_long_diam_mm') * pl.col('age_approx'),
            hue_color_std_interaction      = pl.col('tbp_lv_H') * pl.col('tbp_lv_color_std_mean'),
            lesion_severity_index          = (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_eccentricity')) / 3,
            shape_complexity_index         = pl.col('border_complexity') + pl.col('lesion_shape_index'),
            color_contrast_index           = pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL') + pl.col('tbp_lv_deltaLBnorm'),
        )
        .with_columns(
            log_lesion_area                = (pl.col('tbp_lv_areaMM2') + 1).log(),
            normalized_lesion_size         = pl.col('clin_size_long_diam_mm') / pl.col('age_approx'),
            mean_hue_difference            = (pl.col('tbp_lv_H') + pl.col('tbp_lv_Hext')) / 2,
            std_dev_contrast               = ((pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2) / 3).sqrt(),
            color_shape_composite_index    = (pl.col('tbp_lv_color_std_mean') + pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_symm_2axis')) / 3,
            lesion_orientation_3d          = pl.arctan2(pl.col('tbp_lv_y'), pl.col('tbp_lv_x')),
            overall_color_difference       = (pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL')) / 3,
        )
        .with_columns(
            symmetry_perimeter_interaction = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_perimeterMM'),
            comprehensive_lesion_index     = (pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_eccentricity') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_symm_2axis')) / 4,
            color_variance_ratio           = pl.col('tbp_lv_color_std_mean') / pl.col('tbp_lv_stdLExt'),
            border_color_interaction       = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color'),
            border_color_interaction_2     = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color') / (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color')),
            size_color_contrast_ratio      = pl.col('clin_size_long_diam_mm') / pl.col('tbp_lv_deltaLBnorm'),
            age_normalized_nevi_confidence = pl.col('tbp_lv_nevi_confidence') / pl.col('age_approx'),
            age_normalized_nevi_confidence_2 = (pl.col('clin_size_long_diam_mm')**2 + pl.col('age_approx')**2).sqrt(),
            color_asymmetry_index          = pl.col('tbp_lv_radial_color_std_max') * pl.col('tbp_lv_symm_2axis'),
        )
        .with_columns(
            volume_approximation_3d        = pl.col('tbp_lv_areaMM2') * (pl.col('tbp_lv_x')**2 + pl.col('tbp_lv_y')**2 + pl.col('tbp_lv_z')**2).sqrt(),
            color_range                    = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs() + (pl.col('tbp_lv_A') - pl.col('tbp_lv_Aext')).abs() + (pl.col('tbp_lv_B') - pl.col('tbp_lv_Bext')).abs(),
            shape_color_consistency        = pl.col('tbp_lv_eccentricity') * pl.col('tbp_lv_color_std_mean'),
            border_length_ratio            = pl.col('tbp_lv_perimeterMM') / (2 * np.pi * (pl.col('tbp_lv_areaMM2') / np.pi).sqrt()),
            age_size_symmetry_index        = pl.col('age_approx') * pl.col('clin_size_long_diam_mm') * pl.col('tbp_lv_symm_2axis'),
            index_age_size_symmetry        = pl.col('age_approx') * pl.col('tbp_lv_areaMM2') * pl.col('tbp_lv_symm_2axis'),
        )
        .with_columns(
            ((pl.col(col) - pl.col(col).mean().over('patient_id')) / (pl.col(col).std().over('patient_id') + err)).alias(f'{col}_patient_norm') for col in (num_cols + new_num_cols)
        )
        .with_columns(
            count_per_patient = pl.col('isic_id').count().over('patient_id'),
        )
        .with_columns(
            pl.col(cat_cols).cast(pl.Categorical)
        )
        .to_pandas()
    )

In [6]:
def preprocess(df_train, df_test):
    global cat_cols

    encoder = OneHotEncoder(sparse_output=False, dtype=np.int32, handle_unknown='ignore')
    encoder.fit(df_train[cat_cols])

    new_cat_cols = [f'onehot_{i}' for i in range(len(encoder.get_feature_names_out()))]

    df_train[new_cat_cols] = encoder.transform(df_train[cat_cols])
    df_train[new_cat_cols] = df_train[new_cat_cols].astype('category')

    df_test[new_cat_cols] = encoder.transform(df_test[cat_cols])
    df_test[new_cat_cols] = df_test[new_cat_cols].astype('category')


    for col in cat_cols:
        feature_cols.remove(col)

    feature_cols.extend(new_cat_cols)
    cat_cols = new_cat_cols

    return df_train, df_test

In [7]:
# df_selfclean = pd.read_csv('/content/ISIC_2024_Challenge_SelfClean_Scores.csv')
# df_selfclean.drop(columns=['irrelevant_ranking', 'label_error_ranking'], inplace=True)
# df_selfclean.head()

In [8]:
# df_train_merged = df_train.merge(df_selfclean, on=["isic_id", "patient_id"])


In [9]:
# count = df_train_merged[(df_train_merged['irrelevant_score'] > 0.99834) & (df_train_merged['target'] == 0)].shape[0]

# print(f"Number of rows where 'irrelevant_score' is more than 0.95 and 'target' is 1: {count}")

In [10]:
# df_train = pd.read_csv("/kaggle/input/isic-2024-challenge/train-metadata.csv")
# df_test = pd.read_csv("/kaggle/input/isic-2024-challenge/test-metadata.csv")

TEST_HDF5_FILE_PATH = '/content/test-image.hdf5'
err = 1e-5
id_col = 'isic_id'

train_path = '/content/train-metadata.csv'
test_path = '/content/test-metadata.csv'
df_train = read_data(train_path)
df_train = df_train[~df_train['isic_id'].isin(bad_ids)].reset_index(drop=True)
df_test = read_data(test_path)

# df_train = df_train.merge(df_selfclean, on=["isic_id", "patient_id"])
# df_train = df_train[(df_train['target'] == 1) | (df_train['irrelevant_score'] <= 0.99834)].reset_index(drop=True)



# df_train = pd.read_csv("/content/train-metadata.csv")
# df_train = df_train[~df_train['isic_id'].isin(bad_ids)].reset_index(drop=True)


num_folds = 5

gkf = GroupKFold(n_splits=num_folds)

df_train["fold"] = -1
for idx, (train_idx, val_idx) in enumerate(gkf.split(df_train, df_train["target"], groups=df_train["patient_id"])):
    df_train.loc[val_idx, "fold"] = idx

# Add summary
fold_summary = df_train.groupby("fold")["patient_id"].nunique().to_dict()
total_patients = df_train["patient_id"].nunique()

print(f"Fold Summary (patients per fold):")
for fold, count in fold_summary.items():
    if fold != -1:  # Exclude the initialization value
        print(f"Fold {fold}: {count} patients")
print(f"Total patients: {total_patients}")

Fold Summary (patients per fold):
Fold 0: 206 patients
Fold 1: 209 patients
Fold 2: 209 patients
Fold 3: 209 patients
Fold 4: 209 patients
Total patients: 1042


In [11]:
df_train, df_test = preprocess(df_train, df_test)
print(f"\t– TRAIN DATAFRAME SHAPE: {df_train.shape}");
print(f"\t– TEST DATAFRAME SHAPE: {df_test.shape}");

print("\n... READS DATA COMPLETE ...\n")

	– TRAIN DATAFRAME SHAPE: (400915, 223)
	– TEST DATAFRAME SHAPE: (3, 211)

... READS DATA COMPLETE ...



In [12]:

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [13]:
# df_train.isna().sum()

In [14]:
# df_train.dropna(inplace=True)

In [15]:
# df_train.head()

In [16]:
df_train.shape

(400915, 223)

In [17]:
# df_train[norm_cols] = df_train[norm_cols].apply(lambda col: col.fillna(col.mean()))
# df_test[norm_cols] = df_test[norm_cols].apply(lambda col: col.fillna(col.mean()))


In [18]:
df_train.shape

(400915, 223)

In [19]:
 df_train[df_train['isic_id'] == 'ISIC_9454701']

,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,tbp_lv_Hext,tbp_lv_L,tbp_lv_Lext,tbp_lv_areaMM2,tbp_lv_area_perim_ratio,tbp_lv_color_std_mean,tbp_lv_deltaA,tbp_lv_deltaB,tbp_lv_deltaL,tbp_lv_deltaLB,tbp_lv_deltaLBnorm,tbp_lv_eccentricity,tbp_lv_location,tbp_lv_location_simple,tbp_lv_minorAxisMM,tbp_lv_nevi_confidence,tbp_lv_norm_border,tbp_lv_norm_color,tbp_lv_perimeterMM,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence,lesion_size_ratio,lesion_shape_index,hue_contrast,luminance_contrast,lesion_color_difference,border_complexity,color_uniformity,position_distance_3d,perimeter_to_area_ratio,area_to_perimeter_ratio,lesion_visibility_score,combined_anatomical_site,symmetry_border_consistency,consistency_symmetry_border,color_consistency,consistency_color,size_age_interaction,hue_color_std_interaction,lesion_severity_index,shape_complexity_index,color_contrast_index,log_lesion_area,normalized_lesion_size,mean_hue_difference,std_dev_contrast,color_shape_composite_index,lesion_orientation_3d,overall_color_difference,symmetry_perimeter_interaction,comprehensive_lesion_index,color_variance_ratio,border_color_interaction,border_color_interaction_2,size_color_contrast_ratio,age_normalized_nevi_confidence,age_normalized_nevi_confidence_2,color_asymmetry_index,volume_approximation_3d,color_range,shape_color_consistency,border_length_ratio,age_size_symmetry_index,index_age_size_symmetry,age_approx_patient_norm,clin_size_long_diam_mm_patient_norm,tbp_lv_A_patient_norm,tbp_lv_Aext_patient_norm,tbp_lv_B_patient_norm,tbp_lv_Bext_patient_norm,tbp_lv_C_patient_norm,tbp_lv_Cext_patient_norm,tbp_lv_H_patient_norm,tbp_lv_Hext_patient_norm,tbp_lv_L_patient_norm,tbp_lv_Lext_patient_norm,tbp_lv_areaMM2_patient_norm,tbp_lv_area_perim_ratio_patient_norm,tbp_lv_color_std_mean_patient_norm,tbp_lv_deltaA_patient_norm,tbp_lv_deltaB_patient_norm,tbp_lv_deltaL_patient_norm,tbp_lv_deltaLB_patient_norm,tbp_lv_deltaLBnorm_patient_norm,tbp_lv_eccentricity_patient_norm,tbp_lv_minorAxisMM_patient_norm,tbp_lv_nevi_confidence_patient_norm,tbp_lv_norm_border_patient_norm,tbp_lv_norm_color_patient_norm,tbp_lv_perimeterMM_patient_norm,tbp_lv_radial_color_std_max_patient_norm,tbp_lv_stdL_patient_norm,tbp_lv_stdLExt_patient_norm,tbp_lv_symm_2axis_patient_norm,tbp_lv_symm_2axis_angle_patient_norm,tbp_lv_x_patient_norm,tbp_lv_y_patient_norm,tbp_lv_z_patient_norm,lesion_size_ratio_patient_norm,lesion_shape_index_patient_norm,hue_contrast_patient_norm,luminance_contrast_patient_norm,lesion_color_difference_patient_norm,border_complexity_patient_norm,color_uniformity_patient_norm,position_distance_3d_patient_norm,perimeter_to_area_ratio_patient_norm,area_to_perimeter_ratio_patient_norm,lesion_visibility_score_patient_norm,symmetry_border_consistency_patient_norm,consistency_symmetry_border_patient_norm,color_consistency_patient_norm,consistency_color_patient_norm,size_age_interaction_patient_norm,hue_color_std_interaction_patient_norm,lesion_severity_index_patient_norm,shape_complexity_index_patient_norm,color_contrast_index_patient_norm,log_lesion_area_patient_norm,normalized_lesion_size_patient_norm,mean_hue_difference_patient_norm,std_dev_contrast_patient_norm,color_shape_composite_index_patient_norm,lesion_orientation_3d_patient_norm,overall_color_difference_patient_norm,symmetry_perimeter_interaction_patient_norm,comprehensive_lesion_index_patient_norm,color_variance_ratio_patient_norm,border_color_interaction_patient_norm,border_color_interaction_2_patient_norm,size_color_contrast_ratio_patient_norm,age_normalized_nevi_confidence_patient_norm,age_normalized_nevi_confidence_2_patient_norm,color_asymmetry_index_patient_norm,v

# Load meta data / review

In [20]:
# Set the HDF5 file path
TRAIN_HDF5_FILE_PATH = '/content/train-image.hdf5'

# are we scoring?
scoring = False
#check length of test data to see if we are scoring....
test_length = len(pd.read_csv("/content/test-metadata.csv"))
if test_length > 3:
    scoring = True

if not scoring:
    if full_train_only_when_scoring:
        df_train = df_train.head(quick_train_record_count)

print("\nOriginal Dataset Summary:")
print(f"Total number of samples: {len(df_train)}")
print(f"Number of unique patients: {df_train['patient_id'].nunique()}")

original_positive_cases = df_train['target'].sum()
original_total_cases = len(df_train)
original_positive_ratio = original_positive_cases / original_total_cases

print(f"Number of positive cases: {original_positive_cases}")
print(f"Number of negative cases: {original_total_cases - original_positive_cases}")
print(f"Ratio of negative to positive cases: {(original_total_cases - original_positive_cases) / original_positive_cases:.2f}:1")


Original Dataset Summary:
Total number of samples: 400915
Number of unique patients: 1042
Number of positive cases: 393
Number of negative cases: 400522
Ratio of negative to positive cases: 1019.14:1


# Downsample Negatives / Keep All Positives
* Keeping just 1% of negatives!
* We only use a small subset of the data in each fold for training...

In [21]:
#keep all positives
df_target_1 = df_train[df_train['target'] == 1]
# df_target_1_oversampled = pd.concat([df_target_1] * 3, ignore_index=True)

#just use 1% of negatives
df_target_0 = df_train[df_train['target'] == 0].sample(frac=0.01, random_state=42)

df_train_balanced = pd.concat([df_target_1, df_target_0], ignore_index=True)
# df_train_balanced = df_train_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Print balanced dataset summary
print("Balanced Dataset Summary:")
print(f"Total number of samples: {len(df_train)}")
print(f"Number of unique patients: {df_train['patient_id'].nunique()}")

positive_cases = df_train_balanced['target'].sum()
total_cases = len(df_train_balanced)
positive_ratio = positive_cases / total_cases

print(f"Number of positive cases: {positive_cases}")
print(f"Number of negative cases: {total_cases - positive_cases}")
print(f"New ratio of negative to positive cases: {(total_cases - positive_cases) / positive_cases:.2f}:1")

Balanced Dataset Summary:
Total number of samples: 400915
Number of unique patients: 1042
Number of positive cases: 393
Number of negative cases: 4005
New ratio of negative to positive cases: 10.19:1


In [22]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Initialize the scaler
scaler = StandardScaler()
# Fit the scaler on the training data and transform it
df_train_balanced[feature_cols] = scaler.fit_transform(df_train_balanced[feature_cols])

# Transform the specified feature columns in the test data using the same scaler
df_test[feature_cols] = scaler.transform(df_test[feature_cols])

# ImageNet Setup for Training

In [23]:
import torch.nn.functional as F
from torch.nn import Parameter

def gem(x, p=3, eps=1e-4):
    return F.avg_pool2d(x.clamp(min=eps), (x.size(-2), x.size(-1))).pow(1.0 / p)

class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6, p_trainable=False):
        super(GeM, self).__init__()
        if p_trainable:
            self.p = Parameter(torch.ones(1) * p)
        else:
            self.p = p
        self.eps = eps


    def forward(self, x):
        ret = gem(x, p=self.p, eps=self.eps)
        return ret


def get_activation(activ_name: str="relu"):
    """"""
    act_dict = {
        "relu": nn.ReLU(inplace=True),
        "leakyReLU" : nn.LeakyReLU(negative_slope=0.01, inplace = True),
        "tanh": nn.Tanh(),
        "sigmoid": nn.Sigmoid(),
        "identity": nn.Identity()}
    if activ_name in act_dict:
        return act_dict[activ_name]
    else:
        raise NotImplementedError


class Conv2dBNActiv(nn.Module):
    """Conv2d -> (BN ->) -> Activation"""

    def __init__(
        self, in_channels, out_channels,
        kernel_size, stride, padding,
        bias=False, use_bn=True, activ="relu"
    ):
        """"""
        super(Conv2dBNActiv, self).__init__()
        layers = []
        layers.append(nn.Conv2d(
            in_channels, out_channels,
            kernel_size, stride, padding, bias=bias))
        if use_bn:
            layers.append(nn.BatchNorm2d(out_channels))

        layers.append(get_activation(activ))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        """Forward"""
        return self.layers(x)


class SpatialAttentionBlock(nn.Module):
    """Spatial Attention for (C, H, W) feature maps"""

    def __init__(
        self, in_channels,
        out_channels_list,
    ):
        """Initialize"""
        super(SpatialAttentionBlock, self).__init__()
        self.n_layers = len(out_channels_list)
        channels_list = [in_channels] + out_channels_list
        assert self.n_layers > 0
        assert channels_list[-1] == 1

        for i in range(self.n_layers - 1):
            in_chs, out_chs = channels_list[i: i + 2]
            layer = Conv2dBNActiv(in_chs, out_chs, 3, 1, 1, activ="relu")
            setattr(self, f"conv{i + 1}", layer)

        in_chs, out_chs = channels_list[-2:]
        layer = Conv2dBNActiv(in_chs, out_chs, 3, 1, 1, activ="sigmoid")
        setattr(self, f"conv{self.n_layers}", layer)

    def forward(self, x):
        """Forward"""
        h = x
        for i in range(self.n_layers):
            h = getattr(self, f"conv{i + 1}")(h)

        h = h * x
        return h

# class MaskGuidedAttention(nn.Module):
#     def __init__(self, in_dim, projected_dim, dropout_rate=0.1, num_heads=8, mask_influence=2.0):
#         super(MaskGuidedAttention, self).__init__()
#         self.projector = nn.Linear(in_dim, projected_dim)
#         self.multihead_attn = nn.MultiheadAttention(embed_dim=projected_dim, num_heads=num_heads, dropout=dropout_rate)
#         self.gamma = nn.Parameter(torch.zeros(1) * 0.5)
#         self.dropout = nn.Dropout(dropout_rate)
#         self.alpha = nn.Parameter(torch.ones(1) * mask_influence)  # Learnable parameter to modulate mask influence
#         self.norm = nn.LayerNorm(projected_dim)  # Layer normalization

#     def forward(self, x, mask):
#         # Reduce mask dimensions to match x
#         mask = torch.mean(mask, dim=[2, 3], keepdim=False)  # [batch_size, sequence_length]
#         mask = mask.unsqueeze(-1)  # [batch_size, sequence_length, 1]

#         # Reshape mask to match the sequence length (seq_len, batch, 1)
#         mask = mask.permute(1, 0, 2)  # [sequence_length, batch_size, 1]

#         # Project x
#         # x = self.projector(x)

#         # Apply multi-head attention
#         attn_output, attn_weights = self.multihead_attn(x, x, x)
#         attn_output = self.norm(attn_output)  # Apply LayerNorm

#         # Expand or repeat mask if necessary to match the sequence length of attn_output
#         if attn_output.size(0) != mask.size(0):
#             mask = F.interpolate(mask.permute(1, 2, 0), size=attn_output.size(0)).permute(2, 0, 1)

#         # Apply soft masking
#         attn_output = attn_output * (self.alpha * mask)  # Element-wise multiplication with attention output
#         attn_output = self.dropout(attn_output)

#         # Residual connection
#         out = self.gamma * attn_output + x
#         return out

class MaskGuidedAttention(nn.Module):
    def __init__(self, in_dim, dropout_rate=0.0, num_heads=8, mask_influence=1.0):
        super(MaskGuidedAttention, self).__init__()
        self.num_heads = num_heads
        self.in_dim = in_dim
        self.head_dim = in_dim // num_heads
        assert self.head_dim * num_heads == in_dim, "in_dim must be divisible by num_heads"

        self.W_Q = nn.Linear(in_dim, in_dim)
        self.W_K = nn.Linear(in_dim, in_dim)
        self.W_V = nn.Linear(in_dim, in_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.gamma = nn.Parameter(torch.tensor(0.25))
        self.alpha = nn.Parameter(torch.tensor(mask_influence))

    def forward(self, x, mask):
        # x: [seq_len, batch_size, in_dim]
        # mask: [batch_size, channels, height, width]
        seq_length, batch_size, _ = x.size()

        # Reshape x to [batch_size, seq_length, in_dim]
        x = x.permute(1, 0, 2)  # [batch_size, seq_length, in_dim]

        # Compute spatial dimensions from seq_length
        x_height = x_width = int(math.sqrt(seq_length))
        if x_height * x_width != seq_length:
            raise ValueError(f"Computed spatial dimensions ({x_height}x{x_width}) do not match seq_length ({seq_length}).")

        # Resize mask to match x's spatial dimensions
        mask = F.interpolate(mask, size=(x_height, x_width), mode='bilinear', align_corners=False)  # [batch_size, channels, x_height, x_width]

        # If mask has multiple channels, reduce to single channel
        if mask.size(1) > 1:
            mask = mask.mean(dim=1, keepdim=True)  # [batch_size, 1, x_height, x_width]

        # Flatten mask to [batch_size, seq_length]
        mask = mask.view(batch_size, -1)

        # Normalize and prepare the mask
        mask = (mask - mask.mean(dim=1, keepdim=True)) / (mask.std(dim=1, keepdim=True) + 1e-5)
        mask = mask.unsqueeze(1).unsqueeze(2)  # [batch_size, 1, 1, seq_length]
        mask = mask.expand(-1, self.num_heads, seq_length, -1)  # [batch_size, num_heads, seq_length, seq_length]

        # Linear projections for Q, K, V
        Q = self.W_Q(x)
        K = self.W_K(x)
        V = self.W_V(x)

        # Reshape for multi-head attention
        Q = Q.view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        K = K.view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        V = V.view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)

        # Compute attention scores
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)

        # Integrate mask into attention scores
        attn_scores = attn_scores + (self.alpha * mask)

        # Compute attention weights
        attn_weights = F.softmax(attn_scores, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Compute attention output
        attn_output = torch.matmul(attn_weights, V)

        # Concatenate heads
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_length, self.in_dim)

        # Residual connection
        out = self.gamma * attn_output + x

        # Permute back to original shape [seq_len, batch_size, in_dim]
        out = out.permute(1, 0, 2)

        return out



class SpatialAttentionBlock2(nn.Module):
    def __init__(self, in_channels):
        super(SpatialAttentionBlock2, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size=7, padding=3)
        self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=1)  # Added to match original channels
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        attn = torch.cat([avg_out, max_out], dim=1)
        attn = self.conv1(attn)
        attn = self.sigmoid(attn)

        # Apply the attention mask to the original input
        return self.conv2(attn * x)

class EnhancedSegmentationHead(nn.Module):
    def __init__(self, in_channels, mid_channels=288, out_channels=1, dropout_prob=0.1):
        super(EnhancedSegmentationHead, self).__init__()

        # Downsample block 1
        self.conv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1)
        self.norm1 = nn.GroupNorm(32, mid_channels)  # Using GroupNorm instead of InstanceNorm
        self.mish1 = nn.Mish(inplace=True)
        self.dropout1 = nn.Dropout2d(0.15)

        # Downsample block 2
        self.conv2 = nn.Conv2d(mid_channels, mid_channels, kernel_size=3, padding=1)
        self.norm2 = nn.GroupNorm(32, mid_channels)
        self.mish2 = nn.Mish(inplace=True)
        self.dropout2 = nn.Dropout2d(0.05)

        # Upsample block 1
        self.upsample1 = nn.ConvTranspose2d(mid_channels, mid_channels // 2, kernel_size=2, stride=2)
        self.norm3 = nn.GroupNorm(16, mid_channels // 2)
        self.mish3 = nn.Mish(inplace=True)
        self.dropout3 = nn.Dropout2d(0.05)

        # Upsample block 2
        self.upsample2 = nn.ConvTranspose2d(mid_channels // 2, mid_channels // 4, kernel_size=2, stride=2)
        self.norm4 = nn.GroupNorm(8, mid_channels // 4)
        self.mish4 = nn.Mish(inplace=True)
        self.dropout4 = nn.Dropout2d(0.1)

        # Spatial Attention
        self.attention = SpatialAttentionBlock2(in_channels // 8)

        # Final convolution layer to reduce channels
        self.conv_out = nn.Conv2d(72, out_channels, kernel_size=1)
        self.dropout5 = nn.Dropout2d(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Downsampling path
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.mish1(x)
        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.norm2(x)
        x = self.mish2(x)
        x = self.dropout2(x)

        # Upsampling path
        x = self.upsample1(x)
        x = self.norm3(x)
        x = self.mish3(x)
        x = self.dropout3(x)

        x = self.upsample2(x)
        x = self.norm4(x)
        x = self.mish4(x)
        x = self.attention(x)  # Apply attention before final conv layer

        # Output
        out = self.conv_out(x)
        out = self.dropout5(out)
        out = self.sigmoid(out)  # Final activation without dropout to ensure stability

        return out



In [24]:
# import torch.nn as nn
# import timm

# class ISICModel(nn.Module):
#     def __init__(self, backbone='tiny_vit_21m_224.dist_in22k_ft_in1k', num_classes=2, pretrained=False, freeze_base_model=False):
#         super(ISICModel, self).__init__()

#         # Encoder setup with the specified backbone
#         self.encoder = timm.create_model(
#             backbone,
#             pretrained=pretrained,
#         )

#         # Freeze the encoder if required
#         if freeze_base_model:
#             self.freeze_encoder(True)

#         self.nb_fts = self.encoder.num_features
#         self.gap = nn.AdaptiveAvgPool2d(1)
#         self.spatial_attention = SpatialAttentionBlock(in_channels=self.nb_fts, out_channels_list=[self.nb_fts // 2, 1])

#         self.dropout = nn.Dropout2d(0.0)

# #         self.segmentation_head = nn.Sequential(
# #             UNetBlock(self.nb_fts, self.nb_fts // 2),
# #             nn.ConvTranspose2d(self.nb_fts // 2, self.nb_fts // 4, kernel_size=2, stride=2),
# #             UNetBlock(self.nb_fts // 4, self.nb_fts // 4),
# #             nn.Conv2d(self.nb_fts // 4, 1, kernel_size=1),
# #             nn.Dropout2d(0.4),
# #             nn.Sigmoid()
# #         )

# #         self.segmentation_head = nn.Sequential(
# #         nn.Conv2d(self.nb_fts, 1, kernel_size=1),
# #         nn.Sigmoid()
# #     )

#         self.segmentation_head = ComplexSegmentationHeadWithDropout(in_channels=self.nb_fts)
#         self.gem_pooling = GeM(p=3, p_trainable=True)
#         self.mask_guided_attention = MaskGuidedAttention(in_dim=self.nb_fts, projected_dim=self.nb_fts, dropout_rate=0.25)


# #         self.head = nn.Sequential(
# #                 SpatialAttentionBlock(self.nb_fts, [self.nb_fts ,1]),
# #                 nn.AdaptiveAvgPool2d(output_size=1),
# #                 nn.Flatten(start_dim=1),
# #                 nn.Linear(self.nb_fts, 128),
# #                 nn.LayerNorm(128),
# #                 nn.Mish(inplace=True),
# #                 nn.Dropout(0.5),
# #                 nn.Linear(128, 2))

# #         Classification head with the specified number of classes
#         self.head = nn.Sequential(
#             nn.Linear(self.nb_fts, 128),
#             nn.LayerNorm(128),
#             nn.Mish(),
#             nn.Dropout(0.25),
#             nn.Linear(128, num_classes),
#         )

#     def forward(self, x):
#         x = self.encoder.forward_features(x)
#         x = self.spatial_attention(x)
#         x = self.dropout(x)
#         seg_mask = self.segmentation_head(x)

#         # Classification Path
#         cls_x = x.flatten(2).permute(2, 0, 1)  # Reshape to [seq_len, batch, nb_fts] for attention
#         cls_x = self.mask_guided_attention(cls_x, seg_mask)
#         cls_x = self.gem_pooling(x)
#         cls_x = cls_x.flatten(1)
#         cls_x = self.head(cls_x)

#         return cls_x

#     def freeze_encoder(self, flag):
#         for param in self.encoder.parameters():
#             param.requires_grad = not flag

In [25]:
import torch.nn as nn
import timm

class ISICModel(nn.Module):
    def __init__(self,
                 backbone='tiny_vit_21m_224.dist_in22k_ft_in1k',
                 num_classes=2,
                 pretrained=False,
                 freeze_base_model=False):
        super(ISICModel, self).__init__()

        # Encoder setup with the specified backbone
        self.encoder = timm.create_model(
            backbone,
            pretrained=pretrained,
        )

        # Freeze the encoder if required
        if freeze_base_model:
            self.freeze_encoder(True)

        self.nb_fts = self.encoder.num_features
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.spatial_attention = SpatialAttentionBlock(in_channels=self.nb_fts, out_channels_list=[self.nb_fts // 2, 1])
        self.dropout = nn.Dropout2d(0.0)

        self.segmentation_head = EnhancedSegmentationHead(in_channels=self.nb_fts)
        self.gem_pooling = GeM(p=3, p_trainable=True)
        self.mask_guided_attention = MaskGuidedAttention(
            in_dim=self.nb_fts,
            dropout_rate=0.0,
            num_heads=8,
            mask_influence=1.00
        )

        # Embedding layers for categorical features
#         self.embeddings = nn.ModuleList([
#             nn.Embedding(num_categories, min(50, (num_categories + 1) // 2))
#             for num_categories in categorical_cardinalities
#         ])

        # Tabular feature processing network
        self.tabular_net = nn.Sequential(
            nn.Linear(124, 512),  # Assuming 82 tabular features
            nn.BatchNorm1d(512),
            nn.SiLU(),  # Activation changed to Mish
            nn.Dropout(0.3),  # Dropout increased to 0.5
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.SiLU()  # Activation changed to Mish
            # nn.Dropout(0.3),  # Dropout increased to 0.5
        )

        # Combined classification head
        self.head = nn.Sequential(
            nn.Linear(self.nb_fts + 128, 128),  # Concatenate image features and tabular features
            # nn.BatchNorm1d(128),
            # nn.Mish(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes),
        )

    def forward(self, image, tabular_data):
            # Image Path
            x = self.encoder.forward_features(image)
            x = self.spatial_attention(x)
            x = self.dropout(x)
            seg_mask = self.segmentation_head(x)

            # Classification Path
            cls_x = x.flatten(2).permute(2, 0, 1)  # Reshape to [seq_len, batch, nb_fts] for attention
            cls_x = self.mask_guided_attention(cls_x, seg_mask)
            cls_x = self.gem_pooling(x)
            cls_x = cls_x.flatten(1)

            # Tabular Path
            tabular_feat = self.tabular_net(tabular_data)

            # Concatenate Image and Tabular Features
            combined_features = torch.cat((cls_x, tabular_feat), dim=1)

            # Final Classification
            output = self.head(combined_features)

            return output, seg_mask

    def freeze_encoder(self, flag):
        for param in self.encoder.parameters():
            param.requires_grad = not flag

model = ISICModel()

# Separate the parameters into different groups
segmentation_params = list(model.segmentation_head.parameters())
classification_params = list(model.head.parameters())
encoder_params = list(model.encoder.parameters())
attention_params = list(model.spatial_attention.parameters()) + \
                   list(model.gem_pooling.parameters()) + \
                   list(model.mask_guided_attention.parameters())



In [26]:
def setup_isic_model(backbone='tiny_vit_21m_224.dist_in22k_ft_in1k', num_classes=2, freeze_base_model=False, pretrained=True):
    model = ISICModel(backbone=backbone, num_classes=num_classes, pretrained=pretrained, freeze_base_model=freeze_base_model)
    return model.to(device)

def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for name, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

# Data Loading / Image Augmentation

In [27]:
# class ISICDataset(Dataset):
#     def __init__(self, hdf5_file, isic_ids, targets=None, transform=None):
#         self.hdf5_file = hdf5_file
#         self.isic_ids = isic_ids
#         self.targets = targets
#         self.transform = transform

#     def get_labels(self):
#         return self.targets

#     def __len__(self):
#         return len(self.isic_ids)

#     def __getitem__(self, idx):
#         with h5py.File(self.hdf5_file, 'r') as f:
#             img_bytes = f[self.isic_ids[idx]][()]

#         img = Image.open(io.BytesIO(img_bytes))
#         img = np.array(img)  # Convert PIL Image to numpy array

#         if self.transform:
#             transformed = self.transform(image=img)
#             img = transformed['image']

#         if self.targets is not None:
#             target = self.targets[idx]
#         else:
#             target = torch.tensor(-1)  # Dummy target for test set

#         return img, target

# # Prepare augmentation
# aug_transform = A.Compose([

#     A.RandomRotate90(),
#     A.Flip(),
# #     A.ShiftScaleRotate(shift_limit=0.0, scale_limit=0.15, rotate_limit=90, p=0.5),
#     A.RandomBrightnessContrast(brightness_limit=0.18, contrast_limit=0.12, p=0.5),
# #     A.OpticalDistortion(distort_limit=0.5, shift_limit=0.0, p=0.7),

#     A.HueSaturationValue(hue_shift_limit=3, sat_shift_limit=5, val_shift_limit=1, p=0.5),

# #     A.ElasticTransform(alpha=0.2, sigma=6.0, alpha_affine=20.0, p=0.5),
#     A.Resize(224, 224),
#     A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ToTensorV2(),
# ])

# base_transform = A.Compose([
#     A.Resize(224, 224),
#     A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ToTensorV2(),
# ])

In [28]:
# aug_transform = A.Compose([

#     A.Transpose(p=0.5),
#     A.VerticalFlip(p=0.5),
#     A.HorizontalFlip(p=0.5),
#     A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.75),
#     A.OneOf([
#         A.MotionBlur(blur_limit=5),
#         A.MedianBlur(blur_limit=5),
#         A.GaussianBlur(blur_limit=5),
#         A.GaussNoise(var_limit=(5.0, 20.0)),
#     ], p=0.7),

#     A.OneOf([
#         A.OpticalDistortion(distort_limit=1.0),
#         A.GridDistortion(num_steps=5, distort_limit=1.),
#         A.ElasticTransform(alpha=3),
#     ], p=0.7),

#     A.CLAHE(clip_limit=4.0, p=0.7),
#     A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
#     A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
#     A.Resize(224, 224),
#     A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ToTensorV2(),
# ])

In [29]:
class ISICDataset(Dataset):
    def __init__(self, hdf5_file, isic_ids, tabular_features=None, targets=None, transform=None):
        self.hdf5_file = hdf5_file
        self.isic_ids = isic_ids
        self.tabular_features = tabular_features  # Add tabular features here
        self.targets = targets
        self.transform = transform

    def get_labels(self):
        return self.targets

    def __len__(self):
        return len(self.isic_ids)

    def __getitem__(self, idx):
        with h5py.File(self.hdf5_file, 'r') as f:
            img_bytes = f[self.isic_ids[idx]][()]

        img = Image.open(io.BytesIO(img_bytes))
        img = np.array(img)  # Convert PIL Image to numpy array

        if self.transform:
            transformed = self.transform(image=img)
            img = transformed['image']

        tabular_feat = None
        if self.tabular_features is not None:
            tabular_feat = self.tabular_features[idx]  # Get the tabular features for this index

        if self.targets is not None:
            target = self.targets[idx]
        else:
            target = torch.tensor(-1)  # Dummy target for test set

        return img, tabular_feat, target  # Return image, tabular features, and target

# Prepare Augmentations
aug_transform = A.Compose([

    A.RandomRotate90(),
    A.Flip(),
    A.ShiftScaleRotate(shift_limit=0.0, scale_limit=0.15, rotate_limit=90, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.18, contrast_limit=0.12, p=0.5),
    A.OpticalDistortion(distort_limit=0.5, shift_limit=0.0, p=0.7),

    A.HueSaturationValue(hue_shift_limit=3, sat_shift_limit=5, val_shift_limit=1, p=0.5),

    A.ElasticTransform(alpha=0.2, sigma=6.0, p=0.5),
    A.Resize(224, 224, interpolation=cv2.INTER_CUBIC),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

base_transform = A.Compose([
    A.Resize(224, 224,interpolation=cv2.INTER_CUBIC),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

<ipython-input-29-744a7e03a2e0>:41: DeprecationWarning: Flip is deprecated. Consider using HorizontalFlip, VerticalFlip, RandomRotate90 or D4.
  A.Flip(),


In [30]:
# aug_transform = A.Compose([

#     A.RandomRotate90(),
#     A.Flip(),
#     A.ShiftScaleRotate(shift_limit=0.0, scale_limit=0.15, rotate_limit=90, p=0.5),
#     A.RandomBrightnessContrast(brightness_limit=0.18, contrast_limit=0.12, p=0.5),
#     A.OpticalDistortion(distort_limit=0.5, shift_limit=0.0, p=0.7),

#     A.HueSaturationValue(hue_shift_limit=3, sat_shift_limit=5, val_shift_limit=1, p=0.5),

#     A.ElasticTransform(alpha=0.2, sigma=6.0, p=0.5),
#     A.Resize(224, 224),
#     A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ToTensorV2(),
# ])

# Visualize image augmentation

In [31]:
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

def visualize_augmentations_positive(dataset, num_samples=3, num_augmentations=5, figsize=(20, 10)):
    # Find positive samples
    positive_samples = []
    for i in range(len(dataset)):
        _, label = dataset[i]
        if label == 1:  # Assuming 1 is the positive class
            positive_samples.append(i)

        if len(positive_samples) == num_samples:
            break

    if len(positive_samples) < num_samples:
        print(f"Warning: Only found {len(positive_samples)} positive samples.")

    fig, axes = plt.subplots(num_samples, num_augmentations + 1, figsize=figsize)
    fig.suptitle("Original and Augmented Versions of Positive Samples", fontsize=16)

    for sample_num, sample_idx in enumerate(positive_samples):
        # Get a single sample
        original_image, label = dataset[sample_idx]

        # If the image is already a tensor (due to ToTensorV2 in the transform), convert it back to numpy
        if isinstance(original_image, torch.Tensor):
            original_image = original_image.permute(1, 2, 0).numpy()

        # Reverse the normalization
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        original_image = (original_image * std + mean) * 255
        original_image = original_image.astype(np.uint8)

        # Display original image
        axes[sample_num, 0].imshow(original_image)
        axes[sample_num, 0].axis('off')
        axes[sample_num, 0].set_title("Original", fontsize=10)

        # Apply and display augmentations
        for aug_num in range(num_augmentations):
            augmented = dataset.transform(image=original_image)['image']
            # If the result is a tensor, convert it back to numpy
            if isinstance(augmented, torch.Tensor):
                augmented = augmented.permute(1, 2, 0).numpy()
            # Reverse the normalization
            augmented = (augmented * std + mean) * 255
            augmented = augmented.astype(np.uint8)

            axes[sample_num, aug_num + 1].imshow(augmented)
            axes[sample_num, aug_num + 1].axis('off')
            axes[sample_num, aug_num + 1].set_title(f"Augmented {aug_num + 1}", fontsize=10)

    plt.tight_layout()
    plt.show()

augtest_dataset = ISICDataset(
    hdf5_file=TRAIN_HDF5_FILE_PATH,
    isic_ids=df_train['isic_id'].values,
    targets=df_train['target'].values,
    transform=aug_transform,
)

# visualize_augmentations_positive(augtest_dataset)

# Scoring code from https://www.kaggle.com/code/metric/isic-pauc-abovetpr

In [32]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80) -> float:

    del solution[row_id_column_name]
    del submission[row_id_column_name]

    # rescale the target. set 0s to 1s and 1s to 0s (since sklearn only has max_fpr)
    v_gt = abs(np.asarray(solution.values)-1)

    # flip the submissions to their compliments
    v_pred = -1.0*np.asarray(submission.values)

    max_fpr = abs(1-min_tpr)

    # using sklearn.metric functions: (1) roc_curve and (2) auc
    fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=None)
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError("Expected min_tpr in range [0, 1), got: %r" % min_tpr)

    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)

    return(partial_auc)

# Train / CV

In [33]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        if isinstance(alpha, (float, int)):  # Removed 'long' as it's not needed in Python 3
            self.alpha = torch.tensor([alpha, 1 - alpha])
        elif isinstance(alpha, list):
            self.alpha = torch.tensor(alpha)
        else:
            self.alpha = alpha
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim() > 2:
            input = input.view(input.size(0), input.size(1), -1)  # N,C,H,W -> N,C,H*W
            input = input.transpose(1, 2)    # N,C,H*W -> N,H*W,C
            input = input.contiguous().view(-1, input.size(2))   # N,H*W,C -> N*H*W,C
        target = target.view(-1, 1)

        # Updated to include dim argument
        logpt = F.log_softmax(input, dim=-1)
        logpt = logpt.gather(1, target)
        logpt = logpt.view(-1)
        pt = logpt.exp()

        if self.alpha is not None:
            if self.alpha.type() != input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0, target.view(-1))
            logpt = logpt * at

        loss = -1 * (1 - pt) ** self.gamma * logpt
        if self.size_average:
            return loss.mean()
        else:
            return loss.sum()

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class CombinedLoss(nn.Module):
    def __init__(self, gamma=0.25, smooth=1e-6):
        super(CombinedLoss, self).__init__()
        self.dice_loss = DiceLoss(smooth)
        self.cross_entropy = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 1.25]),label_smoothing=0.1) # weight=torch.tensor([1.0, 1.25]
        self.gamma = gamma

    def forward(self, logits, seg_mask, class_targets):
        # Self-supervised segmentation part
        pseudo_targets = generate_pseudo_targets(seg_mask)
        seg_loss = self.dice_loss(seg_mask, pseudo_targets)

        # Classification part
        class_loss = self.cross_entropy(logits, class_targets)

        # Combine losses
        loss = self.gamma * seg_loss + 0.75 * class_loss
        return loss

class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, logits, targets):
        probs = torch.sigmoid(logits)
        num = probs * targets  # Element-wise multiplication between the predicted and true masks
        num = 2 * torch.sum(num, dim=(1, 2, 3))  # Sum over all dimensions except batch size

        den = probs + targets  # Element-wise addition
        den = torch.sum(den, dim=(1, 2, 3))  # Sum over all dimensions except batch size

        dice = (num + self.smooth) / (den + self.smooth)
        return 1 - dice.mean()  # Dice Loss is 1 - Dice Coefficient

def generate_pseudo_targets(seg_mask):
    # Generate pseudo-targets from the segmentation mask (this can be the model's own predictions)
    return torch.sigmoid(seg_mask)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, SequentialSampler, RandomSampler
from torch.autograd import Variable

# def train_evaluate(model, train_loader, val_loader, criterion, optimizer, scheduler, device):
#     scaler = GradScaler()

#     # Training phase
#     model.train()
#     for inputs, targets in tqdm(train_loader, desc="Training"):
#         inputs, targets = inputs.to(device), targets.to(device)
#         optimizer.zero_grad(set_to_none=True)

#         with autocast():
#             outputs = model(inputs)
#             loss = criterion(outputs, targets)

#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()

#     # Evaluation phase
#     model.eval()
#     val_targets, val_outputs = [], []
#     with torch.no_grad(), autocast():
#         for inputs, targets in tqdm(val_loader, desc="Evaluating"):
#             inputs, targets = inputs.to(device), targets.to(device)
#             outputs = model(inputs)
#             val_targets.append(targets.cpu())
#             val_outputs.append(outputs.softmax(dim=1)[:, 1].cpu())

#     scheduler.step()
#     return torch.cat(val_targets).numpy(), torch.cat(val_outputs).numpy()

def train_evaluate(model, train_loader, val_loader, criterion, optimizer, scheduler, device):
    scaler = GradScaler()  # Updated from the deprecated version

    # Training phase
    model.train()
    train_loss = 0.0
    for images, tabular_data, class_targets in tqdm(train_loader, desc="Training"):
        images, tabular_data, class_targets = images.to(device), tabular_data.to(device).float(), class_targets.to(device)
        optimizer.zero_grad(set_to_none=True)

        with autocast():  # Updated to the new autocast syntax
            outputs, seg_mask = model(images, tabular_data)  # Pass both images and tabular data
            loss = criterion(outputs, seg_mask, class_targets)  # Pass seg_mask to criterion

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_value_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)  # Calculate average loss for training

    # Evaluation phase
    model.eval()
    val_loss = 0.0
    val_targets, val_outputs = [], []
    with torch.no_grad(), autocast():  # Updated to the new autocast syntax
        for images, tabular_data, class_targets in tqdm(val_loader, desc="Evaluating"):
            images, tabular_data, class_targets = images.to(device), tabular_data.to(device).float(), class_targets.to(device)
            outputs, seg_mask = model(images, tabular_data)  # Pass both images and tabular data
            loss = criterion(outputs, seg_mask, class_targets)  # Pass seg_mask to criterion
            val_loss += loss.item()
            val_targets.append(class_targets.cpu())
            val_outputs.append(outputs.softmax(dim=1)[:, 1].cpu())

    avg_val_loss = val_loss / len(val_loader)  # Calculate average loss for validation

    scheduler.step()

    return torch.cat(val_targets).numpy(), torch.cat(val_outputs).numpy(), avg_train_loss, avg_val_loss

def cross_validation_train(df_train, num_folds, num_epochs, hdf5_file_path, aug_transform, base_transform, device):
    # Define the combined loss function for Cross Entropy and Tversky Loss
    criterion = CombinedLoss(gamma=0.25).to(device)
    best_overall_targets, best_overall_outputs = None, None  # Best results across all folds

    for fold in range(num_folds):
        print(f"\nFold {fold + 1}/{num_folds}")

        # Split data for the current fold
        train_df = df_train[df_train['fold'] != fold]
        val_df = df_train[df_train['fold'] == fold]

        # Create datasets and data loaders
        train_dataset = ISICDataset(hdf5_file_path, train_df['isic_id'].values, train_df[feature_cols].values, train_df['target'].values, aug_transform)
        val_dataset = ISICDataset(hdf5_file_path, val_df['isic_id'].values, val_df[feature_cols].values, val_df['target'].values, base_transform)

        labels = train_dataset.get_labels()
        class_weights = torch.tensor(compute_class_weight(class_weight="balanced", classes=np.unique(labels), y=labels))

        samples_weights = class_weights[labels]
        print(class_weights)
        sampler = WeightedRandomSampler(weights=samples_weights, num_samples=len(samples_weights), replacement=True)

        train_loader = DataLoader(train_dataset, batch_size=128, shuffle=sampler, num_workers=8, pin_memory=True)
        val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=8, pin_memory=True)

        # Initialize model, optimizer, and scheduler once per fold
        model = setup_isic_model().to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=0.00005, weight_decay=1e-6)
        # optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.9, weight_decay=1e-5)
        # Define the optimizer with different learning rates for each parameter group
        # optimizer = torch.optim.AdamW([
        #     {'params': segmentation_params, 'lr': 0.003},  # Learning rate for segmentation task
        #     {'params': classification_params, 'lr': 0.0009}  # Learning rate for classification task
        # ], weight_decay=1e-5)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

        best_score = float('-inf')  # Initialize the best score to negative infinity
        best_model_path = f'model_fold_{fold + 1}_best.pth'  # Path to save the best model for this fold

        best_val_targets, best_val_outputs = None, None  # Best targets and outputs for this fold

        for epoch in range(num_epochs):
            print(f"\nEpoch {epoch + 1}/{num_epochs}")

            print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, "
                  f"Train Pos Ratio: {train_df['target'].mean():.2%}, Val Pos Ratio: {val_df['target'].mean():.2%}")

            # Train and evaluate
            val_targets, val_outputs, avg_train_loss, avg_val_loss = train_evaluate(
                model, train_loader, val_loader, criterion, optimizer, scheduler, device)

            print(f'Epoch {epoch + 1} - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

            # Create DataFrames with row_id for scoring
            solution_df = pd.DataFrame({'target': val_targets, 'row_id': range(len(val_targets))})
            submission_df = pd.DataFrame({'prediction': val_outputs, 'row_id': range(len(val_outputs))})
            fold_score = score(solution_df, submission_df, 'row_id')
            print(f'Fold {fold + 1}, Epoch {epoch + 1} pAUC Score: {fold_score:.4f}')

            # Save the model and best results if this is the best score for this fold
            if fold_score > best_score:
                best_score = fold_score
                best_val_targets = val_targets  # Save the best targets
                best_val_outputs = val_outputs  # Save the best outputs
                torch.save(model.state_dict(), best_model_path)
                print(f"New best model saved for fold {fold + 1} with pAUC Score: {best_score:.4f}")

        # After all epochs for the current fold, store the best results
        if best_val_targets is not None and best_val_outputs is not None:
            if best_overall_targets is None and best_overall_outputs is None:
                best_overall_targets = best_val_targets
                best_overall_outputs = best_val_outputs
            else:
                best_overall_targets = np.concatenate([best_overall_targets, best_val_targets])
                best_overall_outputs = np.concatenate([best_overall_outputs, best_val_outputs])

    print(f'\nBest models saved for each fold.')

    # Return the best accumulated targets and outputs for final evaluation
    return np.array(best_overall_targets), np.array(best_overall_outputs)


# Set up CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Perform cross-validation training and get the accumulated results
all_val_targets, all_val_outputs = cross_validation_train(df_train_balanced, num_folds, num_epochs, TRAIN_HDF5_FILE_PATH, aug_transform, base_transform, device)

Using device: cuda

Fold 1/5
tensor([0.5488, 5.6230], dtype=torch.float64)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
<ipyth


Epoch 1/50
Train: 3475, Val: 923, Train Pos Ratio: 8.89%, Val Pos Ratio: 9.10%


Training:   0%|          | 0/28 [00:00<?, ?it/s]<ipython-input-35-89ba35077f3a>:52: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Updated to the new autocast syntax
Training: 100%|██████████| 28/28 [01:04<00:00,  2.31s/it]
<ipython-input-35-89ba35077f3a>:70: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast():  # Updated to the new autocast syntax
Evaluating: 100%|██████████| 4/4 [00:08<00:00,  2.10s/it]


Epoch 1 - Train Loss: 0.4322, Val Loss: 0.3787
Fold 1, Epoch 1 pAUC Score: 0.1222


...

Training:   0%|          | 0/28 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
<ipython-input-35-89ba35077f3a>:52: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Updated to the new autocast syntax
Training: 100%|██████████| 28/28 [01:00<00:00,  2.14s/it]
<ipython-input-35-89ba35077f3a>:70: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast():  # Updat

Epoch 38 - Train Loss: 0.2963, Val Loss: 0.3011
Fold 3, Epoch 38 pAUC Score: 0.1699

Epoch 39/50
Train: 3519, Val: 879, Train Pos Ratio: 9.07%, Val Pos Ratio: 8.42%


Training:   0%|          | 0/28 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
<ipython-input-35-89ba35077f3a>:52: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Updated to the new autocast syntax
Training: 100%|██████████| 28/28 [01:01<00:00,  2.21s/it]
<ipython-input-35-89ba35077f3a>:70: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast():  # Updat

Epoch 39 - Train Loss: 0.2980, Val Loss: 0.3017
Fold 3, Epoch 39 pAUC Score: 0.1702

Epoch 40/50
Train: 3519, Val: 879, Train Pos Ratio: 9.07%, Val Pos Ratio: 8.42%


Training:   0%|          | 0/28 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
<ipython-input-35-89ba35077f3a>:52: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Updated to the new autocast syntax
Training: 100%|██████████| 28/28 [01:02<00:00,  1.28it/s]

In [ ]:
# from google.colab import files
# files.download('model_fold_1_best.pth')
# # files.download('model_fold_2_best.pth')
# # files.download('model_fold_3_best.pth')
# # files.download('model_fold_4_best.pth')
# # files.download('model_fold_5_best.pth')

In [ ]:
!cp /content/model_fold_1_best.pth /content/drive/MyDrive/Kaggle

cp: cannot create regular file '/content/drive/MyDrive/Kaggle': No such file or directory


In [ ]:
!cp /content/model_fold_2_best.pth /content/drive/MyDrive/Kaggle

In [ ]:
!cp /content/model_fold_3_best.pth /content/drive/MyDrive/Kaggle

In [ ]:
!cp /content/model_fold_4_best.pth /content/drive/MyDrive/Kaggle

In [ ]:
!cp /content/model_fold_5_best.pth /content/drive/MyDrive/Kaggle

In [ ]:
# Final overall evaluation
print("\nFinal Overall Evaluation:")

# Calculate the official pAUC score
solution_df = pd.DataFrame({'target': all_val_targets, 'row_id': range(len(all_val_targets))})
submission_df = pd.DataFrame({'prediction': all_val_outputs, 'row_id': range(len(all_val_outputs))})
official_score = score(solution_df, submission_df, 'row_id')
print(f'Overall pAUC Score: {official_score:.4f}')

# Generate and print classification report
binary_predictions = binarize(np.array(all_val_outputs).reshape(-1, 1), threshold=0.5).reshape(-1)
report = classification_report(all_val_targets, binary_predictions, target_names=['Class 0', 'Class 1'])
print("\nOverall Classification Report:")
print(report)

# Print specific metrics for Class 1
report_dict = classification_report(all_val_targets, binary_predictions, target_names=['Class 0', 'Class 1'], output_dict=True)
print(f"\nClass 1 Metrics:")
print(f"Precision: {report_dict['Class 1']['precision']:.4f}")
print(f"Recall: {report_dict['Class 1']['recall']:.4f}")
print(f"F1-score: {report_dict['Class 1']['f1-score']:.4f}")


Final Overall Evaluation:
Overall pAUC Score: 0.1691

Overall Classification Report:
              precision    recall  f1-score   support

     Class 0       0.96      0.98      0.97      4005
     Class 1       0.74      0.63      0.68       393

    accuracy                           0.95      4398
   macro avg       0.85      0.81      0.83      4398
weighted avg       0.94      0.95      0.95      4398


Class 1 Metrics:
Precision: 0.7433
Recall: 0.6336
F1-score: 0.6841


In [ ]:
# Final Overall Evaluation:
# Overall pAUC Score: 0.1693

# Overall Classification Report:
#               precision    recall  f1-score   support

#      Class 0       0.96      0.98      0.97      4005
#      Class 1       0.74      0.63      0.68       393

#     accuracy                           0.95      4398
#    macro avg       0.85      0.80      0.83      4398
# weighted avg       0.94      0.95      0.95      4398


# Class 1 Metrics:
# Precision: 0.7395
# Recall: 0.6285
# # F1-score: 0.6795

In [ ]:

# Final Overall Evaluation:
# Overall pAUC Score: 0.1643

# Overall Classification Report:
#               precision    recall  f1-score   support

#      Class 0       0.98      0.92      0.95      4005
#      Class 1       0.52      0.85      0.65       393

#     accuracy                           0.92      4398
#    macro avg       0.75      0.89      0.80      4398
# weighted avg       0.94      0.92      0.93      4398


# Class 1 Metrics:
# Precision: 0.5226
# Recall: 0.8550
# F1-score: 0.6486

In [ ]:
# Final Overall Evaluation: 1.5 with tab
# Overall pAUC Score: 0.1496

# Overall Classification Report:
#               precision    recall  f1-score   support

#      Class 0       0.99      0.85      0.91      4005
#      Class 1       0.37      0.87      0.51       393

#     accuracy                           0.85      4398
#    macro avg       0.68      0.86      0.71      4398
# weighted avg       0.93      0.85      0.88      4398


# Class 1 Metrics:
# Precision: 0.3654
# Recall: 0.8702
# F1-score: 0.5147

In [ ]:
# Final Overall Evaluation:
# Overall pAUC Score: 0.1524 1.25 with tab

# Overall Classification Report:
#               precision    recall  f1-score   support

#      Class 0       0.98      0.90      0.94      4005
#      Class 1       0.45      0.83      0.58       393

#     accuracy                           0.89      4398
#    macro avg       0.71      0.86      0.76      4398
# weighted avg       0.93      0.89      0.91      4398


# Class 1 Metrics:
# Precision: 0.4464
# Recall: 0.8270
# F1-score: 0.5798

In [ ]:
# Final Overall Evaluation: CE 1
# Overall pAUC Score: 0.1468

# Overall Classification Report:
#               precision    recall  f1-score   support

#      Class 0       0.98      0.88      0.93      4005
#      Class 1       0.41      0.82      0.54       393

#     accuracy                           0.88      4398
#    macro avg       0.69      0.85      0.74      4398
# weighted avg       0.93      0.88      0.89      4398


# Class 1 Metrics:
# Precision: 0.4076
# Recall: 0.8193
# F1-score: 0.5444

In [ ]:
# Final Overall Evaluation: CE 1.5
# Overall pAUC Score: 0.1481

# Overall Classification Report:
#               precision    recall  f1-score   support

#      Class 0       0.98      0.88      0.93      4005
#      Class 1       0.40      0.82      0.53       393

#     accuracy                           0.87      4398
#    macro avg       0.69      0.85      0.73      4398
# weighted avg       0.93      0.87      0.89      4398


# Class 1 Metrics:
# Precision: 0.3973
# Recall: 0.8168
# F1-score: 0.5346

In [ ]:
# Final Overall Evaluation: CE:2 1
# Overall pAUC Score: 0.1434

# Overall Classification Report:
#               precision    recall  f1-score   support

#      Class 0       0.98      0.84      0.91      4005
#      Class 1       0.35      0.87      0.50       393

#     accuracy                           0.85      4398
#    macro avg       0.67      0.86      0.71      4398
# weighted avg       0.93      0.85      0.87      4398


# Class 1 Metrics:
# Precision: 0.3526
# Recall: 0.8677
# F1-score: 0.5015

In [ ]:
# Final Overall Evaluation:  CE 3:1
# Overall pAUC Score: 0.1411

# Overall Classification Report:
#               precision    recall  f1-score   support

#      Class 0       0.99      0.84      0.91      4005
#      Class 1       0.35      0.87      0.50       393

#     accuracy                           0.84      4398
#    macro avg       0.67      0.86      0.70      4398
# weighted avg       0.93      0.84      0.87      4398


# Class 1 Metrics:
# Precision: 0.3518
# Recall: 0.8728
# F1-score: 0.5015

In [ ]:
from google.colab import runtime
runtime.unassign()

# Inference Code
* There are some duplicate definitions / includes here to make copying to other notebooks easier

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from tqdm import tqdm
import h5py
import timm
from torchvision import transforms
from PIL import Image
import io
import albumentations as A
from albumentations.pytorch import ToTensorV2

epoch_for_preds = num_epochs
model_path = ""

class ISICDataset(Dataset):
    def __init__(self, hdf5_file, isic_ids, targets=None, transform=None):
        self.hdf5_file = h5py.File(hdf5_file, 'r')  # Keep file open
        self.isic_ids = isic_ids
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.isic_ids)

    def __getitem__(self, idx):
        img_bytes = self.hdf5_file[self.isic_ids[idx]][()]
        img = Image.open(io.BytesIO(img_bytes))
        img = np.array(img)

        if self.transform:
            transformed = self.transform(image=img)
            img = transformed['image']

        target = self.targets[idx] if self.targets is not None else torch.tensor(-1)
        return img, target

    def __del__(self):
        self.hdf5_file.close()  # Ensure file is closed when object is destroyed

# Define the albumentations transformation
base_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

def setup_isic_model(backbone='tiny_vit_21m_224.dist_in22k_ft_in1k', num_classes=2, freeze_base_model=False, pretrained=True):
    model = ISICModel(backbone=backbone, num_classes=num_classes, pretrained=pretrained, freeze_base_model=freeze_base_model)
    return model.to(device)

def load_models(folds, device):
    models = []
    for fold in folds:
        model = setup_isic_model().to(device)
        model.load_state_dict(torch.load(f'{model_path}model_fold_{fold+1}_best.pth', map_location=device))
        model.eval()
        models.append(model)
    return models

@torch.no_grad()  # Apply no_grad to the entire function
def ensemble_predict(models, test_loader, device):
    all_predictions = []
    for inputs, _ in tqdm(test_loader, desc="Predicting"):
        inputs = inputs.to(device)
        fold_predictions = torch.stack([model(inputs).softmax(dim=1)[:, 1] for model in models])
        avg_predictions = fold_predictions.mean(dim=0)
        all_predictions.extend(avg_predictions.cpu().numpy())
    return all_predictions


# Generate out-of-fold predictions for Train
* Only done if not being submitted

In [ ]:
def generate_oof_predictions(df_train, folds, hdf5_file_path, transform):
    oof_predictions = np.zeros(len(df_train))
    model_filenames = [''] * len(df_train)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    models = load_models(folds, device)

    for fold in folds:
        print(f"Generating predictions for fold {fold + 1}/{num_folds}")
        val_df = df_train[df_train['fold'] == fold].copy()
        val_dataset = ISICDataset(hdf5_file_path, val_df['isic_id'].values, val_df['target'].values, transform)
        val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False, num_workers=4, pin_memory=True)

        fold_predictions = ensemble_predict([models[fold]], val_loader, device)

        oof_predictions[val_df.index] = fold_predictions
        model_filename = f'model_fold_{fold+1}_best.pth'
        for idx in val_df.index:
            model_filenames[idx] = model_filename

    return oof_predictions, model_filenames


if not scoring:
    # Set up CUDA if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Define the number of folds
    folds = [0, 1, 2, 3, 4]

    # Generate out-of-fold predictions
    oof_predictions, model_filenames = generate_oof_predictions(df_train, folds, TRAIN_HDF5_FILE_PATH, base_transform)

    # Create DataFrame for OOF predictions
    oof_df = pd.DataFrame({
        'isic_id': df_train['isic_id'],
        'target': df_train['target'],
        'fold': df_train['fold'],
        'oof_prediction': oof_predictions,
        'model_filename': model_filenames
    })

    # Save OOF predictions to CSV
    oof_df.to_csv('oof_predictions.csv', index=False)
    print("Out-of-fold predictions saved to oof_predictions.csv")
    print(oof_df.head())

# Predict for test

In [ ]:
df_test = pd.read_csv("/kaggle/input/isic-2024-challenge/test-metadata.csv")
TEST_HDF5_FILE_PATH = '/kaggle/input/isic-2024-challenge/test-image.hdf5'

# Set up CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# folds to use for pred
folds = [0, 1, 2, 3, 4, 5]

models = load_models(folds, device)

# Prepare your test dataset
test_dataset = ISICDataset(
    hdf5_file=TEST_HDF5_FILE_PATH,
    isic_ids=df_test['isic_id'].values,
    transform=base_transform,
)

# Create test data loader
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False, num_workers=4, pin_memory=True)

# Run predictions
predictions = ensemble_predict(models, test_loader, device)

# Create submission DataFrame
submission_df = pd.DataFrame({
    'isic_id': df_test['isic_id'],
    'target': predictions
})

# Save predictions to CSV
submission_df.to_csv('submission.csv', index=False)
print("Predictions saved to submission.csv")
print(submission_df.head())